<a href="https://colab.research.google.com/github/shivaansh81001/btc-prediction/blob/main/btc_news_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

from transformers import pipeline
from textblob import TextBlob

In [4]:
sample=Table().read_table('sample_data/cryptopanic_news.csv')


In [5]:
sample.sort("newsDatetime",descending=False)
sample.show(10)

id,title,sourceId,newsDatetime,url,negative,positive,important,liked,disliked,lol,toxic,saved,comments
873,ApeCoin Community Votes to Stay on Ethereum,2,6/9/2022 6:57,https://cryptopanic.com/news/15444990/ApeCoin-Community- ...,0,0,0,0,0,0,0,0,0
875,Terraform Labs Co-Founder Denies Accusations of Fraud,2,6/9/2022 6:03,https://cryptopanic.com/news/15443520/Terraform-Labs-Co- ...,0,0,0,1,0,0,0,0,0
876,Anchor dev claims he warned Do Kwon over unsustainable 2 ...,7,6/9/2022 5:39,https://cryptopanic.com/news/15442676/Anchor-dev-claims- ...,1,0,1,0,0,0,0,0,0
877,"With 12 Key Measures of Prices, Truflation’s Revamped Da ...",11,6/9/2022 5:32,https://cryptopanic.com/news/15442724/With-12-Key-Measur ...,0,0,0,0,0,0,0,0,0
878,'Father' of the Metaverse Neal Stephenson launching Meta ...,7,6/9/2022 5:28,https://cryptopanic.com/news/15442350/Father-of-the-Meta ...,0,1,0,2,0,0,0,0,0
879,Solana launches $100M fund to attract crypto projects fr ...,14,6/9/2022 5:20,https://cryptopanic.com/news/15442294/Solana-launches-10 ...,0,1,0,1,1,0,0,0,0
880,Alibaba Cloud launches service to build NFT marketplaces,13,6/9/2022 5:01,https://cryptopanic.com/news/15442278/Alibaba-Cloud-laun ...,0,2,0,1,0,1,0,0,0
881,"Balderton, Animoca Brands back Request Finance in $5.5 m ...",3,6/9/2022 4:45,https://cryptopanic.com/news/15442253/Balderton-Animoca- ...,0,0,0,0,0,0,0,0,0
882,Optimism Foundation sends $20M to the wrong wallet – OP ...,14,6/9/2022 4:00,https://cryptopanic.com/news/15442200/Optimism-Foundatio ...,1,0,1,3,0,3,0,0,3
883,Creating Utility for Tokenized Derivatives of Proof-of-S ...,10,6/9/2022 3:40,https://cryptopanic.com/news/15442162/Creating-Utility-f ...,0,0,1,0,0,0,0,0,0


In [10]:
def format_date_time(dt):
  temp = dt.split()
  time= temp[1].split(':')
  return '-'.join(temp[0].split('/')[::-1])+' '+time[0].zfill(2)+':'+time[1].zfill(2)+':00'

In [11]:
#test

format_date_time('6/9/2022 5:20')

'2022-9-6 05:20:00'

In [12]:
new_date= sample.apply(format_date_time,"newsDatetime")
sample= sample.with_column('newsDatetime',new_date)


In [14]:
sample= sample.sort("newsDatetime",descending=False)
sample.show(10)

id,title,sourceId,newsDatetime,url,negative,positive,important,liked,disliked,lol,toxic,saved,comments
6435,Dash’s Rise to $800: The Altcoin Displays its Biggest Mo ...,32,2017-1-12 01:55:00,https://cryptopanic.com/news/508188/Dashs-Rise-to-800-Th ...,2,2,0,0,0,0,0,0,0
21763,"IOTA Partners with Cisco, Volkswagen and Others on Data ...",32,2017-1-12 19:35:00,https://cryptopanic.com/news/510877/IOTA-Partners-with-C ...,0,9,2,0,0,0,0,0,0
28311,"Bitcoin Gold Defies Gravity, But Price Rally Looks Weak",30,2017-11-12 15:00:00,https://cryptopanic.com/news/570529/Bitcoin-Gold-Defies- ...,0,1,0,0,0,0,0,0,0
4500,Confusion and Euphoria As Bitcoin Cash Tops $30 Billion,30,2017-12-11 05:45:00,https://cryptopanic.com/news/437504/Confusion-and-Euphor ...,1,1,2,0,1,0,0,0,0
4499,Bitcoin Cash Hard Forks In Bid to Ease Mining Difficulties,30,2017-13-11 22:30:00,https://cryptopanic.com/news/443996/Bitcoin-Cash-Hard-Fo ...,2,2,1,0,0,0,0,1,0
82470,"Ripple Soars to New High Following the Rally of Bitcoin, ...",32,2017-13-12 16:00:00,https://cryptopanic.com/news/litecoin/582070/Ripple-Soar ...,0,7,5,0,0,0,0,0,0
4498,Bitcoin Cash Hard Forks to a Better Difficulty Algorithm,32,2017-14-11 00:01:00,https://cryptopanic.com/news/444245/Bitcoin-Cash-Hard-Fo ...,0,8,5,0,0,0,0,1,0
81452,Ethereum Browser Bug Could Put User Funds At Risk,30,2017-15-12 12:05:00,https://cryptopanic.com/news/ethereum/591668/Ethereum-Br ...,3,0,1,0,0,0,0,0,0
15225,Why are there Discrepancies Between Cryptcurrency Exchan ...,32,2017-15-12 16:10:00,https://cryptopanic.com/news/592155/Why-are-there-Discre ...,0,2,1,0,0,0,0,3,0
81451,How Amazon Could Topple Bitcoin,32,2017-15-12 20:15:00,https://cryptopanic.com/news/ethereum/592926/How-Amazon- ...,6,1,2,0,0,0,0,0,0


In [15]:
sample =sample.where("title",are.containing("Bitcoin")).select('title','newsDatetime')
sample.show(10)

title,newsDatetime
"Bitcoin Gold Defies Gravity, But Price Rally Looks Weak",2017-11-12 15:00:00
Confusion and Euphoria As Bitcoin Cash Tops $30 Billion,2017-12-11 05:45:00
Bitcoin Cash Hard Forks In Bid to Ease Mining Difficulties,2017-13-11 22:30:00
"Ripple Soars to New High Following the Rally of Bitcoin, ...",2017-13-12 16:00:00
Bitcoin Cash Hard Forks to a Better Difficulty Algorithm,2017-14-11 00:01:00
How Amazon Could Topple Bitcoin,2017-15-12 20:15:00
Payment Processor BitPay To Add Support for Bitcoin Cash,2017-19-12 15:00:00
First Successful Bitcoin-Litecoin Cross-Chain Transactio ...,2017-20-11 04:00:00
Speculation of Insider Trading at Coinbase after Surpris ...,2017-20-12 03:35:00
Bitstamp Announces Support for Bitcoin Cash by December,2017-23-11 01:10:00


In [16]:
sentiment_analyzer = pipeline("sentiment-analysis", model="ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [17]:
sentiment_=[]
polarity_=[]
subjectivity_=[]

In [18]:
for news in sample.column("title"):
  result=sentiment_analyzer(news)[0]
  ans =result['label']

  if ans =="positive":
    sentiment_.append(1)
  elif ans =="negative":
    sentiment_.append(-1)
  elif ans =="neutral":
    sentiment_.append(0)
  else:
    print(ans)

  blob=TextBlob(news)

  polarity_.append(round(blob.sentiment.polarity,2))
  subjectivity_.append(round(blob.sentiment.subjectivity,2))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [19]:
print(len(sentiment_),len(polarity_),len(subjectivity_))

11025 11025 11025


In [20]:
sample= sample.with_columns("sentiment",np.array(sentiment_),"polarity",np.array(polarity_),"subjectivity",np.array(subjectivity_))
sample.show(3)

title,newsDatetime,sentiment,polarity,subjectivity
"Bitcoin Gold Defies Gravity, But Price Rally Looks Weak",2017-11-12 15:00:00,1,-0.38,0.62
Confusion and Euphoria As Bitcoin Cash Tops $30 Billion,2017-12-11 05:45:00,-1,0,0
Bitcoin Cash Hard Forks In Bid to Ease Mining Difficulties,2017-13-11 22:30:00,1,-0.29,0.54


In [21]:

sample.to_csv('news_sent_analysis.csv')